In [19]:
# 프로젝트 루트를 Python 경로에 추가
import sys
import os
from pathlib import Path
import asyncio

# 현재 노트북의 위치에서 프로젝트 루트 찾기
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # examples 폴더에서 실행하는 경우
    project_root = current_dir.parent.parent
else:
    # 프로젝트 루트에서 실행하는 경우
    project_root = current_dir

# 프로젝트 루트를 Python 경로에 추가
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 필요한 모듈 가져오기
from typing import List, Dict, Any, Optional
from datetime import datetime
import time
import ray

from quantbt import (
    # Dict Native 전략 시스템
    TradingStrategy,
    BacktestEngine,
    
    # 기본 모듈들
    SimpleBroker, 
    BacktestConfig,
    UpbitDataProvider,
    
    # 주문 관련
    Order, OrderSide, OrderType,
)

from quantbt.ray import (
    RayDataManager, 
    BacktestActor,
    QuantBTEngineAdapter
)

In [2]:
class SimpleSMAStrategy(TradingStrategy):
    """SMA 전략 (Ray 최적화용)
    
    고성능 스트리밍 방식:
    - 지표 계산: Polars 벡터연산 
    - 신호 생성: 행별 스트리밍 처리
    
    매수: 가격이 buy_sma 상회
    매도: 가격이 sell_sma 하회  
    """
    
    def __init__(self, buy_sma: int = 15, sell_sma: int = 30):
        super().__init__(
            name="SimpleSMAStrategy",
            config={
                "buy_sma": buy_sma,
                "sell_sma": sell_sma
            },
            position_size_pct=0.8,  # 80%씩 포지션
            max_positions=1
        )
        self.buy_sma = buy_sma
        self.sell_sma = sell_sma
        
    def _compute_indicators_for_symbol(self, symbol_data):
        """심볼별 이동평균 지표 계산 (Polars 벡터 연산)"""
        
        # 시간순 정렬 확인
        data = symbol_data.sort("timestamp")
        
        # 단순 이동평균 계산
        buy_sma = self.calculate_sma(data["close"], self.buy_sma)
        sell_sma = self.calculate_sma(data["close"], self.sell_sma)
        
        # 지표 컬럼 추가 (중복 방지)
        columns_to_add = []
        
        # buy_sma 컬럼 추가
        buy_sma_name = f"sma_{self.buy_sma}"
        columns_to_add.append(buy_sma.alias(buy_sma_name))
        
        # sell_sma 컬럼 추가 (중복 체크)
        sell_sma_name = f"sma_{self.sell_sma}"
        if sell_sma_name != buy_sma_name:  # 중복이 아닌 경우만 추가
            columns_to_add.append(sell_sma.alias(sell_sma_name))
        
        return data.with_columns(columns_to_add)
    
    def generate_signals_dict(self, current_data: Dict[str, Any]) -> List[Order]:
        """행 데이터 기반 신호 생성"""
        orders = []
        
        if not self.broker:
            return orders
        
        symbol = current_data['symbol']
        current_price = current_data['close']
        
        # SMA 값 가져오기 (같은 값인 경우 하나의 컬럼만 존재)
        buy_sma_name = f'sma_{self.buy_sma}'
        sell_sma_name = f'sma_{self.sell_sma}'
        
        buy_sma = current_data.get(buy_sma_name)
        if buy_sma_name == sell_sma_name:
            sell_sma = buy_sma  # 같은 SMA 값인 경우
        else:
            sell_sma = current_data.get(sell_sma_name)
        
        # 지표가 계산되지 않은 경우 건너뛰기
        if buy_sma is None or sell_sma is None:
            return orders
        
        current_positions = self.get_current_positions()
        
        # 매수 신호: 가격이 buy_sma 상회 + 포지션 없음
        if current_price > buy_sma and symbol not in current_positions:
            portfolio_value = self.get_portfolio_value()
            quantity = self.calculate_position_size(symbol, current_price, portfolio_value)
            
            if quantity > 0:
                order = Order(
                    symbol=symbol,
                    side=OrderSide.BUY,
                    quantity=quantity,
                    order_type=OrderType.MARKET
                )
                orders.append(order)
        
        # 매도 신호: 가격이 sell_sma 하회 + 포지션 있음
        elif current_price < sell_sma and symbol in current_positions and current_positions[symbol] > 0:
            order = Order(
                symbol=symbol,
                side=OrderSide.SELL,
                quantity=current_positions[symbol],
                order_type=OrderType.MARKET
            )
            orders.append(order)
        
        return orders

In [3]:
# 1. Ray 초기화 (runtime_env 설정으로 worker에서 모듈 접근 가능하도록)
if not ray.is_initialized():
    ray.init(
        num_cpus=4,
        object_store_memory=1000000000,  # 1GB
        ignore_reinit_error=True,
        logging_level="ERROR",
        runtime_env={
            "working_dir": str(project_root),
            "py_modules": [str(project_root / "quantbt")]
        }
    )
    print("✅ Ray 클러스터 초기화 완료")

✅ Ray 클러스터 초기화 완료


(BacktestActor pid=1006015) 캐시에 데이터가 없어 새로 로딩: ['KRW-BTC']_2024-01-01 00:00:00_2024-12-31 00:00:00_1d
(BacktestActor pid=1006015) 캐시에 데이터가 없어 새로 로딩: ['KRW-BTC']_2024-01-01 00:00:00_2024-12-31 00:00:00_1d
(BacktestActor pid=1006015) 캐시에 데이터가 없어 새로 로딩: ['KRW-BTC']_2024-01-01 00:00:00_2024-12-31 00:00:00_1d
(BacktestActor pid=1006015) 캐시에 데이터가 없어 새로 로딩: ['KRW-BTC']_2024-01-01 00:00:00_2024-12-31 00:00:00_1d


In [4]:
# 2. 백테스트 기본 설정
config = BacktestConfig(
    symbols=["KRW-BTC"],
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 12, 31),
    timeframe="1d",
    initial_cash=10_000_000,
    commission_rate=0.0,
    slippage_rate=0.0,
    save_portfolio_history=False
)
print("✅ 백테스트 설정 완료")

✅ 백테스트 설정 완료


In [5]:
# 3. RayDataManager 생성
data_manager = RayDataManager.remote()
print("✅ RayDataManager 생성 완료")

✅ RayDataManager 생성 완료


In [6]:
# 4. 데이터 미리 로딩 (한 번만 실행)
print("📊 실제 데이터 로딩 중...")

data_ref = await data_manager.load_real_data.remote(
    symbols=config.symbols,
    start_date=config.start_date,
    end_date=config.end_date,
    timeframe=config.timeframe
)

📊 실제 데이터 로딩 중...


In [7]:
# 5. 캐시 통계 확인
cache_stats = await data_manager.get_cache_stats.remote()
print(f"📈 캐시 통계: {cache_stats['cache_size']}개 데이터, {cache_stats['total_data_size']/1024/1024:,.2f} MB")

📈 캐시 통계: 1개 데이터, 0.02 MB


In [8]:
# 6. BacktestActor들 생성 (RayDataManager 참조와 함께)
num_actors = 4
print(f"\n🎯 {num_actors}개 BacktestActor 생성 중...")

actors = []
for i in range(num_actors):
    actor = BacktestActor.remote(f"actor_{i}", data_manager)
    actors.append(actor)


🎯 4개 BacktestActor 생성 중...


In [9]:
# Actor 초기화
config_dict = {
    'symbols': config.symbols,
    'start_date': config.start_date,
    'end_date': config.end_date,
    'timeframe': config.timeframe,
    'initial_cash': config.initial_cash,
    'commission_rate': config.commission_rate,
    'slippage_rate': config.slippage_rate,
    'save_portfolio_history': config.save_portfolio_history
}

init_results = await asyncio.gather(*[
    actor.initialize_engine.remote(config_dict) for actor in actors
])

In [10]:
successful_actors = sum(init_results)
print(f"✅ BacktestActor 초기화: {successful_actors}/{num_actors}개 성공")

✅ BacktestActor 초기화: 4/4개 성공


In [11]:
# 7. 파라메터 그리드 정의
param_grid = {
    'buy_sma': [10, 15, 20, 25],      # 매수 SMA: 10, 15, 20, 25
    'sell_sma': [25, 30, 35, 40]      # 매도 SMA: 25, 30, 35, 40
}
total_combinations = len(param_grid['buy_sma']) * len(param_grid['sell_sma'])
print(f"✅ 파라메터 그리드 정의 완료: {total_combinations}개 조합")
print(f"   - 매수 SMA: {param_grid['buy_sma']}")
print(f"   - 매도 SMA: {param_grid['sell_sma']}")

✅ 파라메터 그리드 정의 완료: 16개 조합
   - 매수 SMA: [10, 15, 20, 25]
   - 매도 SMA: [25, 30, 35, 40]


In [12]:
# 8. 파라메터 조합 생성
from itertools import product
param_combinations = []
for buy_sma, sell_sma in product(param_grid['buy_sma'], param_grid['sell_sma']):
    param_combinations.append({
        'buy_sma': buy_sma,
        'sell_sma': sell_sma
    })

print(len(param_combinations))

16


In [13]:
# 9. 분산 백테스트 실행
print("분산 백테스트 실행 (제로카피 데이터 공유)")

# Actor별로 작업 분배
tasks = []
for i, params in enumerate(param_combinations):
    actor_idx = i % len(actors)
    actor = actors[actor_idx]
    
    task = actor.execute_backtest.remote(params, SimpleSMAStrategy)
    tasks.append((i, params, task))
    
# 모든 작업 완료 대기
print(f"📊 {total_combinations}개 백테스트 병렬 실행 중...")

분산 백테스트 실행 (제로카피 데이터 공유)
📊 16개 백테스트 병렬 실행 중...


In [14]:
results = []
for i, params, task in tasks:
    try:
        result = await task
        results.append({
            'params': params,
            'result': result,
            'success': True,
            'task_id': i
        })
    except Exception as e:
        print(f"❌ 작업 {i} 실패: {e}")
        results.append({
            'params': params,
            'result': None,
            'success': False,
            'error': str(e),
            'task_id': i
        })

In [15]:
successful_results = [r for r in results if r['success']]
failed_results = [r for r in results if not r['success']]

print(f"✅ 성공한 조합: {len(successful_results)}/{total_combinations}개")
print(f"✅ 성공률: {len(successful_results)/total_combinations*100:.1f}%")

✅ 성공한 조합: 16/16개
✅ 성공률: 100.0%


In [16]:
successful_results[0]

{'params': {'buy_sma': 10, 'sell_sma': 25},
 'result': {'total_return': 1.0334293744037217,
  'sharpe_ratio': 3.273059493217384,
  'max_drawdown': 0.1855409673853017,
  'win_rate': 0.4166666666666667,
  'total_trades': 48,
  'final_portfolio_value': 20334293.74403722,
  'annual_return': 1.03441808938866,
  'volatility': 0.31604011217401906,
  'profit_factor': 3.7379503706721224,
  'execution_time': 0.009536266326904297,
  'success': True,
  'params': {'buy_sma': 10, 'sell_sma': 25},
  'worker_id': 'actor_0'},
 'success': True,
 'task_id': 0}

In [17]:
# 최적 파라메터 찾기
best_result = max(successful_results, 
                    key=lambda x: x['result'].get('sharpe_ratio', -999))

print(f"\n🏆 최적 파라메터:")
print(f"   - 매수 SMA: {best_result['params']['buy_sma']}")
print(f"   - 매도 SMA: {best_result['params']['sell_sma']}")

print(f"\n📈 최고 성과:")
print(f"   - 샤프 비율: {best_result['result'].get('sharpe_ratio', 0):.4f}")
print(f"   - 총 수익률: {best_result['result'].get('total_return', 0):.4f}")

# 성능 통계
sharpe_ratios = [r['result'].get('sharpe_ratio', 0) for r in successful_results]
returns = [r['result'].get('total_return', 0) for r in successful_results]

print(f"\n📊 성능 통계:")
print(f"   - 평균 샤프 비율: {sum(sharpe_ratios)/len(sharpe_ratios):.4f}")
print(f"   - 최고 샤프 비율: {max(sharpe_ratios):.4f}")
print(f"   - 최저 샤프 비율: {min(sharpe_ratios):.4f}")
print(f"   - 평균 수익률: {sum(returns)/len(returns):.4f}")


🏆 최적 파라메터:
   - 매수 SMA: 15
   - 매도 SMA: 35

📈 최고 성과:
   - 샤프 비율: 4.4053
   - 총 수익률: 1.3700

📊 성능 통계:
   - 평균 샤프 비율: 3.7753
   - 최고 샤프 비율: 4.4053
   - 최저 샤프 비율: 3.2220
   - 평균 수익률: 1.1635
